# Project 2: Ames Housing Sale Price Challenge

## Problem Statement
This project sets out to identify as accurate a model, with a limit of 30 variables, as possble for Ames housing sale prices using the corresponding information, using regression techniques, enhanced by applications of feature engineering, feature selection and regularisation.

## Executive Summary

The objective of this project is to create the best regression model to predict housing sale price in Ames, with the goals as stated in the problem statement. The data provided are one set of training data, one set of test data and one set of submission example. With this model, homeowners and real estate agents are able to narrow down the features that would best affect the market value of their properties for investment and holding value. 

Within the training data and test data are 80 variables of different features of the housing with the training data has one extra column of Saleprice for training purposes. Both data require data cleaning and imputation for missing data.

After a careful process of selection and modelling, the Lasso model is found to be the most suitable model with the lowest RMSE scoring. In order to narrow down to 30 variables, Recursive Feature Elimination with Scikit Learn was deployed to ensure the objective was met. 

### This project contains the following notebooks
- 01_Data_cleaning_and_Preprocessing.ipynb (this file) for data cleaning and preprocessing the training and test data.
- 02_Feature_selection_and_modeling.ipynb where features are eliminated and differnt models tested for the best score.


### Contents

- [Data Cleaning](#Data-Cleaning)
 - [Columns with null values](#Columns-with-null-values)
 - [Rename of columns](#Rename-of-columns)
 - [Ordinal Variables Mapping for train set](#Ordinal-Variables-Mapping-for-train-set)
 - [Read in test data for data cleaning](#Read-in-test-data-for-data-cleaning)
 - [Fill in Null Information](#Fill-in-Null-Information)
 - [Ordinal Variables Mapping for test set](#Ordinal-Variables-Mapping-for-test-set)
 
 
 ¶

## Data Cleaning

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, LassoCV, RidgeCV
from sklearn import metrics

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

For a start, the data will be read in after which we will take a deeper look into the features of the dataset (e.g. the dtypes, the values, missing value counts) by breaking it up into 20 columns each time for ease of viewing.

In [2]:
#Reading the data
train = pd.read_csv('../datasets/train.csv')

In [3]:
#Check the headers of dataset
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,NaN,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [4]:
#The shape of the data indicates there are 2051 line items (properties) with 81 variables, including the sale price 
train.shape

(2051, 81)

In [5]:
#Cursory look into the features using pandas profiling
#from pandas_profiling import ProfileReport
#profile = ProfileReport(train, title='Pandas Profiling Report', html={'style':{'full_width':True}})
#profile.to_file(output_file="Profile_Report.html")
#Profile_Report is saved in datasets folder

#### Check first 20 columns

In [6]:
train.iloc[:,:21]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79.0,11449,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,1Story,8,5,2007
2047,785,905377130,30,RL,NaN,12342,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,4,5,1940
2048,916,909253010,50,RL,57.0,7558,Pave,NaN,Reg,Bnk,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1.5Fin,6,6,1928
2049,639,535179160,20,RL,80.0,10400,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,4,5,1956


In [7]:
train.iloc[:,:21].describe(include="all")

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built
count,2051.000000,2.051000e+03,2051.000000,2051,1721.000000,2051.000000,2051,140,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051,2051.000000,2051.000000,2051.000000
unique,NaN,NaN,NaN,7,NaN,NaN,2,2,4,4,3,5,3,28,9,8,5,8,NaN,NaN,NaN
top,NaN,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,NaN,NaN,NaN
freq,NaN,NaN,NaN,1598,NaN,NaN,2044,85,1295,1843,2049,1503,1953,310,1767,2025,1700,1059,NaN,NaN,NaN
mean,1474.033642,7.135900e+08,57.008776,NaN,69.055200,10065.208191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.112140,5.562165,1971.708922
std,843.980841,1.886918e+08,42.824223,NaN,23.260653,6742.488909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.426271,1.104497,30.177889
min,1.000000,5.263011e+08,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1872.000000
25%,753.500000,5.284581e+08,20.000000,NaN,58.000000,7500.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,5.000000,1953.500000
50%,1486.000000,5.354532e+08,50.000000,NaN,68.000000,9430.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,5.000000,1974.000000
75%,2198.000000,9.071801e+08,70.000000,NaN,80.000000,11513.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.000000,6.000000,2001.000000


In [8]:
train.iloc[:,:21].dtypes

Id                int64
PID               int64
MS SubClass       int64
MS Zoning        object
Lot Frontage    float64
Lot Area          int64
Street           object
Alley            object
Lot Shape        object
Land Contour     object
Utilities        object
Lot Config       object
Land Slope       object
Neighborhood     object
Condition 1      object
Condition 2      object
Bldg Type        object
House Style      object
Overall Qual      int64
Overall Cond      int64
Year Built        int64
dtype: object

In [9]:
train.iloc[:,:21].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            2051 non-null   int64  
 1   PID           2051 non-null   int64  
 2   MS SubClass   2051 non-null   int64  
 3   MS Zoning     2051 non-null   object 
 4   Lot Frontage  1721 non-null   float64
 5   Lot Area      2051 non-null   int64  
 6   Street        2051 non-null   object 
 7   Alley         140 non-null    object 
 8   Lot Shape     2051 non-null   object 
 9   Land Contour  2051 non-null   object 
 10  Utilities     2051 non-null   object 
 11  Lot Config    2051 non-null   object 
 12  Land Slope    2051 non-null   object 
 13  Neighborhood  2051 non-null   object 
 14  Condition 1   2051 non-null   object 
 15  Condition 2   2051 non-null   object 
 16  Bldg Type     2051 non-null   object 
 17  House Style   2051 non-null   object 
 18  Overall Qual  2051 non-null 

- Incomplete count for Lot Frontage and Alley

#### Check columns 21-40

In [10]:
train.iloc[:,21:41].describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Year Remod/Add,2051,NaN,NaN,NaN,1984.19,21.0363,1950,1964.5,1993,2004,2010
Roof Style,2051,6,Gable,1619,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Roof Matl,2051,6,CompShg,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Exterior 1st,2051,15,VinylSd,724,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Exterior 2nd,2051,15,VinylSd,721,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mas Vnr Type,2029,4,None,1218,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mas Vnr Area,2029,NaN,NaN,NaN,99.6959,174.963,0,0,0,161,1600
Exter Qual,2051,4,TA,1247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Exter Cond,2051,5,TA,1778,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Foundation,2051,6,PConc,926,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train.iloc[:,21:41].dtypes

Year Remod/Add      int64
Roof Style         object
Roof Matl          object
Exterior 1st       object
Exterior 2nd       object
Mas Vnr Type       object
Mas Vnr Area      float64
Exter Qual         object
Exter Cond         object
Foundation         object
Bsmt Qual          object
Bsmt Cond          object
Bsmt Exposure      object
BsmtFin Type 1     object
BsmtFin SF 1      float64
BsmtFin Type 2     object
BsmtFin SF 2      float64
Bsmt Unf SF       float64
Total Bsmt SF     float64
Heating            object
dtype: object

In [12]:
train.iloc[:,21:41].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year Remod/Add  2051 non-null   int64  
 1   Roof Style      2051 non-null   object 
 2   Roof Matl       2051 non-null   object 
 3   Exterior 1st    2051 non-null   object 
 4   Exterior 2nd    2051 non-null   object 
 5   Mas Vnr Type    2029 non-null   object 
 6   Mas Vnr Area    2029 non-null   float64
 7   Exter Qual      2051 non-null   object 
 8   Exter Cond      2051 non-null   object 
 9   Foundation      2051 non-null   object 
 10  Bsmt Qual       1996 non-null   object 
 11  Bsmt Cond       1996 non-null   object 
 12  Bsmt Exposure   1993 non-null   object 
 13  BsmtFin Type 1  1996 non-null   object 
 14  BsmtFin SF 1    2050 non-null   float64
 15  BsmtFin Type 2  1995 non-null   object 
 16  BsmtFin SF 2    2050 non-null   float64
 17  Bsmt Unf SF     2050 non-null   f

In [13]:
train.iloc[:,21:41].head()

,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating
0,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA
1,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA
2,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA
3,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA
4,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA


Missing line items for the following columns:-
- Mas Vnr Type
- Mas Vnr Area
- Bsmt Qual
- Bsmt Cond
- Bsmt Exposure
- BsmtFin Type 1
- BsmtFin SF 1
- Bsmt Type 2
- Bsmt Unf SF
- Total Bsmt SF

#### Check rows 41-60

In [14]:
train.iloc[:,41:61].head()

,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt
0,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0
1,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0
2,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0
3,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,NaN,BuiltIn,2007.0
4,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Detchd,1957.0


In [15]:
train.iloc[:,41:61].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Heating QC       2051 non-null   object 
 1   Central Air      2051 non-null   object 
 2   Electrical       2051 non-null   object 
 3   1st Flr SF       2051 non-null   int64  
 4   2nd Flr SF       2051 non-null   int64  
 5   Low Qual Fin SF  2051 non-null   int64  
 6   Gr Liv Area      2051 non-null   int64  
 7   Bsmt Full Bath   2049 non-null   float64
 8   Bsmt Half Bath   2049 non-null   float64
 9   Full Bath        2051 non-null   int64  
 10  Half Bath        2051 non-null   int64  
 11  Bedroom AbvGr    2051 non-null   int64  
 12  Kitchen AbvGr    2051 non-null   int64  
 13  Kitchen Qual     2051 non-null   object 
 14  TotRms AbvGrd    2051 non-null   int64  
 15  Functional       2051 non-null   object 
 16  Fireplaces       2051 non-null   int64  
 17  Fireplace Qu  

In [16]:
train.iloc[:,41:61].describe().T

,count,mean,std,min,25%,50%,75%,max
1st Flr SF,2051.0,1164.488055,396.446923,334.0,879.5,1093.0,1405.0,5095.0
2nd Flr SF,2051.0,329.329108,425.671046,0.0,0.0,0.0,692.5,1862.0
Low Qual Fin SF,2051.0,5.512921,51.068870,0.0,0.0,0.0,0.0,1064.0
Gr Liv Area,2051.0,1499.330083,500.447829,334.0,1129.0,1444.0,1728.5,5642.0
Bsmt Full Bath,2049.0,0.427526,0.522673,0.0,0.0,0.0,1.0,3.0
Bsmt Half Bath,2049.0,0.063446,0.251705,0.0,0.0,0.0,0.0,2.0
Full Bath,2051.0,1.577279,0.549279,0.0,1.0,2.0,2.0,4.0
Half Bath,2051.0,0.371039,0.501043,0.0,0.0,0.0,1.0,2.0
Bedroom AbvGr,2051.0,2.843491,0.826618,0.0,2.0,3.0,3.0,8.0
Kitchen AbvGr,2051.0,1.042906,0.209790,0.0,1.0,1.0,1.0,3.0


In [17]:
train.iloc[:,41:61].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Heating QC       2051 non-null   object 
 1   Central Air      2051 non-null   object 
 2   Electrical       2051 non-null   object 
 3   1st Flr SF       2051 non-null   int64  
 4   2nd Flr SF       2051 non-null   int64  
 5   Low Qual Fin SF  2051 non-null   int64  
 6   Gr Liv Area      2051 non-null   int64  
 7   Bsmt Full Bath   2049 non-null   float64
 8   Bsmt Half Bath   2049 non-null   float64
 9   Full Bath        2051 non-null   int64  
 10  Half Bath        2051 non-null   int64  
 11  Bedroom AbvGr    2051 non-null   int64  
 12  Kitchen AbvGr    2051 non-null   int64  
 13  Kitchen Qual     2051 non-null   object 
 14  TotRms AbvGrd    2051 non-null   int64  
 15  Functional       2051 non-null   object 
 16  Fireplaces       2051 non-null   int64  
 17  Fireplace Qu  

Missing line items for the following columns:-
- Bsmt Full Bath
- Bsmt Half Bath
- Garage Type
- Garage Yr Blt

#### Check rows 61-80

In [18]:
train.iloc[:,61:82].head()

,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [19]:
train.iloc[:,61:82].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Garage Finish   1937 non-null   object 
 1   Garage Cars     2050 non-null   float64
 2   Garage Area     2050 non-null   float64
 3   Garage Qual     1937 non-null   object 
 4   Garage Cond     1937 non-null   object 
 5   Paved Drive     2051 non-null   object 
 6   Wood Deck SF    2051 non-null   int64  
 7   Open Porch SF   2051 non-null   int64  
 8   Enclosed Porch  2051 non-null   int64  
 9   3Ssn Porch      2051 non-null   int64  
 10  Screen Porch    2051 non-null   int64  
 11  Pool Area       2051 non-null   int64  
 12  Pool QC         9 non-null      object 
 13  Fence           400 non-null    object 
 14  Misc Feature    65 non-null     object 
 15  Misc Val        2051 non-null   int64  
 16  Mo Sold         2051 non-null   int64  
 17  Yr Sold         2051 non-null   i

In [20]:
train.iloc[:,61:82].describe().T

,count,mean,std,min,25%,50%,75%,max
Garage Cars,2050.0,1.776585,0.764537,0.0,1.0,2.0,2.0,5.0
Garage Area,2050.0,473.671707,215.934561,0.0,319.0,480.0,576.0,1418.0
Wood Deck SF,2051.0,93.833740,128.549416,0.0,0.0,0.0,168.0,1424.0
Open Porch SF,2051.0,47.556802,66.747241,0.0,0.0,27.0,70.0,547.0
Enclosed Porch,2051.0,22.571916,59.845110,0.0,0.0,0.0,0.0,432.0
3Ssn Porch,2051.0,2.591419,25.229615,0.0,0.0,0.0,0.0,508.0
Screen Porch,2051.0,16.511458,57.374204,0.0,0.0,0.0,0.0,490.0
Pool Area,2051.0,2.397855,37.782570,0.0,0.0,0.0,0.0,800.0
Misc Val,2051.0,51.574354,573.393985,0.0,0.0,0.0,0.0,17000.0
Mo Sold,2051.0,6.219893,2.744736,1.0,4.0,6.0,8.0,12.0


In [21]:
train.iloc[:,61:82].dtypes

Garage Finish      object
Garage Cars       float64
Garage Area       float64
Garage Qual        object
Garage Cond        object
Paved Drive        object
Wood Deck SF        int64
Open Porch SF       int64
Enclosed Porch      int64
3Ssn Porch          int64
Screen Porch        int64
Pool Area           int64
Pool QC            object
Fence              object
Misc Feature       object
Misc Val            int64
Mo Sold             int64
Yr Sold             int64
Sale Type          object
SalePrice           int64
dtype: object

Missing items list:
- Garage Finish
- Garage Cars
- Garage Qual
- Garage Cond
- Pool QC
- Fence
- Misc Feature

### Columns with null values

In [22]:
#List and sort the columns with missing values
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum().sort_values(ascending=False)

Pool QC           2042
Misc Feature      1986
Alley             1911
Fence             1651
Fireplace Qu      1000
Lot Frontage       330
Garage Yr Blt      114
Garage Cond        114
Garage Qual        114
Garage Finish      114
Garage Type        113
Bsmt Exposure       58
BsmtFin Type 2      56
Bsmt Cond           55
Bsmt Qual           55
BsmtFin Type 1      55
Mas Vnr Type        22
Mas Vnr Area        22
Bsmt Half Bath       2
Bsmt Full Bath       2
Garage Cars          1
BsmtFin SF 1         1
BsmtFin SF 2         1
Bsmt Unf SF          1
Garage Area          1
Total Bsmt SF        1
dtype: int64

In [23]:
#Displaying columns with at least 1 null value
train[null_columns].head(10)

,Lot Frontage,Alley,Mas Vnr Type,Mas Vnr Area,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Pool QC,Fence,Misc Feature
0,NaN,NaN,BrkFace,289.0,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,0.0,0.0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,NaN,NaN,NaN
1,43.0,NaN,BrkFace,132.0,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,1.0,0.0,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,NaN,NaN,NaN
2,68.0,NaN,None,0.0,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,1.0,0.0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,NaN,NaN,NaN
3,73.0,NaN,None,0.0,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,0.0,0.0,NaN,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,NaN,NaN,NaN
4,82.0,NaN,None,0.0,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,0.0,0.0,NaN,Detchd,1957.0,Unf,2.0,484.0,TA,TA,NaN,NaN,NaN
5,137.0,NaN,None,0.0,TA,TA,No,ALQ,247.0,Rec,713.0,557.0,1517.0,0.0,0.0,Gd,Attchd,1966.0,Fin,2.0,578.0,TA,TA,NaN,NaN,NaN
6,35.0,NaN,BrkFace,82.0,Gd,TA,Gd,GLQ,547.0,Unf,0.0,0.0,547.0,1.0,0.0,NaN,Basment,2005.0,Fin,2.0,525.0,TA,TA,NaN,NaN,NaN
7,NaN,NaN,BrkFace,180.0,TA,TA,No,Rec,1000.0,Unf,0.0,188.0,1188.0,1.0,0.0,NaN,Attchd,1959.0,RFn,2.0,531.0,TA,TA,NaN,MnPrv,NaN
8,NaN,NaN,None,0.0,TA,TA,No,Rec,292.0,Unf,0.0,632.0,924.0,0.0,0.0,NaN,Detchd,1952.0,Unf,1.0,420.0,TA,TA,NaN,MnPrv,Shed
9,70.0,NaN,BrkFace,192.0,Gd,TA,Av,Rec,650.0,Unf,0.0,390.0,1040.0,0.0,1.0,TA,Attchd,1969.0,Unf,2.0,504.0,TA,TA,NaN,NaN,NaN


We will now fill the null values and replacing it with Na in accordance to the data dictionary provided and where applicable the mean or median below.

#### Pool QC

In [24]:
#Replacing the NA values (null) with Na instead as per data dictionary, NA means No Pool
train['Pool QC'] = train['Pool QC'].fillna('Na')

In [25]:
#Check if values are replaced
train['Pool QC'].unique()

array(['Na', 'Fa', 'Gd', 'Ex', 'TA'], dtype=object)

#### Misc Features:

In [26]:
#Replacing the NA values (null) with Na instead as per data dictionary, NA means None
train['Misc Feature'] = train['Misc Feature'].fillna('Na')

In [27]:
#Check if values are replaced
train['Misc Feature'].unique()

array(['Na', 'Shed', 'TenC', 'Gar2', 'Othr', 'Elev'], dtype=object)

#### Alley

In [28]:
#Replacing the NA values (null) with Na instead as per data dictionary, NA means No alley access
train['Alley'] = train['Alley'].fillna('Na')

In [29]:
#Check if values are replaced
train['Alley'].unique()

array(['Na', 'Pave', 'Grvl'], dtype=object)

#### Fence

In [30]:
 #Replacing the NA values (null) with Na instead as per data dictionary, NA means No Fence
train['Fence'] = train['Fence'].fillna('Na')

In [31]:
#Check if values are replaced
train['Fence'].unique()

array(['Na', 'MnPrv', 'GdPrv', 'GdWo', 'MnWw'], dtype=object)

#### Fireplace

In [32]:
#Replacing the NA values (null) with Na instead as per data dictionary, NA means No Fireplace
train['Fireplace Qu'] = train['Fireplace Qu'].fillna('Na')

#### Lot Frontage

In [33]:
#since there are large numbers of null but it seems unlikely for lot frontage to be nil, to replace with mean. 
train['Lot Frontage'] = train['Lot Frontage'].fillna(train['Lot Frontage'].mean())

In [34]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum().sort_values(ascending=False)

Garage Cond       114
Garage Finish     114
Garage Yr Blt     114
Garage Qual       114
Garage Type       113
Bsmt Exposure      58
BsmtFin Type 2     56
Bsmt Qual          55
Bsmt Cond          55
BsmtFin Type 1     55
Mas Vnr Area       22
Mas Vnr Type       22
Bsmt Full Bath      2
Bsmt Half Bath      2
BsmtFin SF 2        1
BsmtFin SF 1        1
Total Bsmt SF       1
Garage Cars         1
Garage Area         1
Bsmt Unf SF         1
dtype: int64

#### Garage Related Columns

In [35]:
garage_cols = (train.filter(regex='Garage'))

In [36]:
garage_cols.isnull().sum()

Garage Type      113
Garage Yr Blt    114
Garage Finish    114
Garage Cars        1
Garage Area        1
Garage Qual      114
Garage Cond      114
dtype: int64

In [37]:
garage_cols

,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond
0,Attchd,1976.0,RFn,2.0,475.0,TA,TA
1,Attchd,1997.0,RFn,2.0,559.0,TA,TA
2,Detchd,1953.0,Unf,1.0,246.0,TA,TA
3,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA
4,Detchd,1957.0,Unf,2.0,484.0,TA,TA
...,...,...,...,...,...,...,...
2046,Attchd,2007.0,Fin,2.0,520.0,TA,TA
2047,Detchd,1961.0,Unf,2.0,539.0,TA,TA
2048,Detchd,1929.0,Unf,2.0,342.0,Fa,Fa
2049,Attchd,1956.0,Unf,1.0,294.0,TA,TA


In [38]:
garage_cols.isnull().sum()

Garage Type      113
Garage Yr Blt    114
Garage Finish    114
Garage Cars        1
Garage Area        1
Garage Qual      114
Garage Cond      114
dtype: int64

In [39]:
train['Garage Type'] = train['Garage Type'].fillna('Na')

In [40]:
#Check if values are replaced
train['Garage Type'].unique()

array(['Attchd', 'Detchd', 'BuiltIn', 'Basment', 'Na', '2Types',
       'CarPort'], dtype=object)

In [41]:
train['Garage Yr Blt'] = train['Garage Yr Blt'].fillna('0')

In [42]:
train['Garage Finish'] = train['Garage Finish'].fillna('Na')

In [43]:
#Check if values are replaced
train['Garage Finish'].unique()

array(['RFn', 'Unf', 'Fin', 'Na'], dtype=object)

In [44]:
train['Garage Qual'] = train['Garage Qual'].fillna('Na')

In [45]:
#Check if values are replaced
train['Garage Qual'].unique()

array(['TA', 'Fa', 'Na', 'Gd', 'Ex', 'Po'], dtype=object)

In [46]:
train['Garage Cond'] = train['Garage Cond'].fillna('Na')

In [47]:
#Check if values are replaced
train['Garage Cond'].unique()

array(['TA', 'Fa', 'Na', 'Po', 'Gd', 'Ex'], dtype=object)

In [48]:
garage_cols.dtypes

Garage Type       object
Garage Yr Blt    float64
Garage Finish     object
Garage Cars      float64
Garage Area      float64
Garage Qual       object
Garage Cond       object
dtype: object

In [49]:
#Check if null values have been replaced
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum().sort_values(ascending=False)

Bsmt Exposure     58
BsmtFin Type 2    56
BsmtFin Type 1    55
Bsmt Cond         55
Bsmt Qual         55
Mas Vnr Area      22
Mas Vnr Type      22
Bsmt Half Bath     2
Bsmt Full Bath     2
Garage Area        1
Garage Cars        1
Total Bsmt SF      1
Bsmt Unf SF        1
BsmtFin SF 2       1
BsmtFin SF 1       1
dtype: int64

In [50]:
#Filter out columns with 'Garage'
garage_cols = (train.filter(regex='Garage'))

In [51]:
garage_cols.isnull().sum()

Garage Type      0
Garage Yr Blt    0
Garage Finish    0
Garage Cars      1
Garage Area      1
Garage Qual      0
Garage Cond      0
dtype: int64

In [52]:
#Row 1712 indicates missing values for Garage Cars and Garage Area since there is a Garage Type. 
garage_cols[garage_cols.isnull().any(axis=1)]

,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond
1712,Detchd,0,Na,NaN,NaN,Na,Na


In [53]:
train['Garage Cars'] = train['Garage Cars'].fillna(train['Garage Cars'].mean())

In [54]:
train['Garage Cars'].unique()

array([2.        , 1.        , 3.        , 0.        , 4.        ,
       5.        , 1.77658537])

In [55]:
train['Garage Area'] = train['Garage Area'].fillna(train['Garage Area'].mean())

In [56]:
#Check if values are replaced
train.iloc[:,61:63].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Garage Finish  2051 non-null   object 
 1   Garage Cars    2051 non-null   float64
dtypes: float64(1), object(1)
memory usage: 32.2+ KB


#### Basement Related Rows

In [57]:
#Filter out basement related rows
bsmt_cols = (train.filter(regex='Bsmt'))

In [58]:
bsmt_cols.isnull().sum()

Bsmt Qual         55
Bsmt Cond         55
Bsmt Exposure     58
BsmtFin Type 1    55
BsmtFin SF 1       1
BsmtFin Type 2    56
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Bsmt Full Bath     2
Bsmt Half Bath     2
dtype: int64

In [59]:
#Change to Na since the corresponding Total Basement Square Foot equals 0, meaning no basement
train['Bsmt Qual'] = train['Bsmt Qual'].fillna('Na')
train['Bsmt Cond'] = train['Bsmt Cond'].fillna('Na')
train['Bsmt Exposure'] = train['Bsmt Exposure'].fillna('Na')
train['BsmtFin Type 1'] = train['BsmtFin Type 1'].fillna('Na')
train['BsmtFin Type 2'] = train['BsmtFin Type 2'].fillna('Na')

In [60]:
#Cross_check to see values have been replaced
bsmt_cols = (train.filter(regex='Bsmt'))
bsmt_cols.isnull().sum()

Bsmt Qual         0
Bsmt Cond         0
Bsmt Exposure     0
BsmtFin Type 1    0
BsmtFin SF 1      1
BsmtFin Type 2    0
BsmtFin SF 2      1
Bsmt Unf SF       1
Total Bsmt SF     1
Bsmt Full Bath    2
Bsmt Half Bath    2
dtype: int64

In [61]:
bsmt_cols[bsmt_cols.isnull().any(axis=1)]

,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath
616,Na,Na,Na,Na,0.0,Na,0.0,0.0,0.0,NaN,NaN
1327,Na,Na,Na,Na,NaN,Na,NaN,NaN,NaN,NaN,NaN


In [62]:
# Change Basement full and half baths to 0 since nothing else is recorded
train['Bsmt Full Bath'] = train['Bsmt Full Bath'].fillna('0')
train['Bsmt Half Bath'] = train['Bsmt Half Bath'].fillna('0')

In [63]:
#Cross check to see values have been replaced
bsmt_cols = (train.filter(regex='Bsmt'))
bsmt_cols.isnull().sum()

Bsmt Qual         0
Bsmt Cond         0
Bsmt Exposure     0
BsmtFin Type 1    0
BsmtFin SF 1      1
BsmtFin Type 2    0
BsmtFin SF 2      1
Bsmt Unf SF       1
Total Bsmt SF     1
Bsmt Full Bath    0
Bsmt Half Bath    0
dtype: int64

In [64]:
#Fill the missing value with mean
train['Total Bsmt SF'] = train['Total Bsmt SF'].fillna(train['Total Bsmt SF'].mean())

In [65]:
#Fill the missing values with median
train['BsmtFin SF 1'] = train['BsmtFin SF 1'].fillna(train['BsmtFin SF 1'].median())

In [66]:
#Fill the missing values with median
train['BsmtFin SF 2'] = train['BsmtFin SF 2'].fillna(train['BsmtFin SF 2'].median())

In [67]:
#Fill the missing values with median
train['Bsmt Unf SF'] = train['Bsmt Unf SF'].fillna(train['Bsmt Unf SF'].median())

In [68]:
#Filter out Bsmt related columns 
bsmt_cols = (train.filter(regex='Bsmt'))
bsmt_cols.isnull().sum()

Bsmt Qual         0
Bsmt Cond         0
Bsmt Exposure     0
BsmtFin Type 1    0
BsmtFin SF 1      0
BsmtFin Type 2    0
BsmtFin SF 2      0
Bsmt Unf SF       0
Total Bsmt SF     0
Bsmt Full Bath    0
Bsmt Half Bath    0
dtype: int64

In [69]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum().sort_values(ascending=False)

Mas Vnr Area    22
Mas Vnr Type    22
dtype: int64

In [70]:
#Change Mas Vnr Type to Na (None)
train['Mas Vnr Type'] = train['Mas Vnr Type'].fillna('Na')

In [71]:
#Change Mas Vnr Area to 0
train['Mas Vnr Area'] = train['Mas Vnr Area'].fillna('0')

### Rename of columns

In [72]:
#Change columns names to lower case and replace space with underscore
train.columns = train.columns.str.lower().str.replace(' ','_')

In [73]:
train.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,69.0552,13517,Pave,Na,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0,0,2,1,3,1,Gd,6,Typ,0,Na,Attchd,1976,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,Na,Na,Na,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0000,11492,Pave,Na,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1,0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,Na,Na,Na,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0000,7922,Pave,Na,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1,0,1,0,3,1,Gd,5,Typ,0,Na,Detchd,1953,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,Na,Na,Na,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0000,9802,Pave,Na,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0,0,2,1,3,1,TA,7,Typ,0,Na,BuiltIn,2007,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,Na,Na,Na,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0000,14235,Pave,Na,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0,0,2,0,3,1,TA,6,Typ,0,Na,Detchd,1957,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,Na,Na,Na,0,3,2010,WD,138500


#### Categorizing the columns into respective types

Nominal:
- ms_subclass
- ms_zoning
- street
- alley
- land_contour
- lot_config	
- neighborhood
- condition_1
- condition_2
- bldg_type
- house_style
- roof_style
- roof_matl
- exterior_1st
- exterior_2nd
- mas_vnr_type
- foundation
- heating
- central_air
- sale_type
- garage_type
- misc_feature

Continuous:
- lot_frontage
- lot_area
- bsmtfin_sf_1
- bsmtfin_sf_2
- bsmt_unf_sf
- total_bsmt_sf
- 1st_flr_sf
- 2nd_flr_sf
- low_qual_fin_sf
- gr_liv_area
- garage_area
- wood_deck_sf
- open_porch_sf
- enclosed_porch
- 3ssn_porch
- screen_porch	
- pool_area
- misc_val
- mas_vnr_area

Discrete:
- year_built
- year_remod/add
- bsmt_full_bath	
- bsmt_half_bath	
- full_bath	
- half_bath	
- bedroom_abvgr	
- kitchen_abvgr 
- totrms_abvgrd
- fireplaces
- yr_sold
- mo_sold
- garage_cars
- garage_yr_blt

Ordinal:
- lot_shape
- overall_qual	
- overall_cond
- exter_qual
- exter_cond
- bsmt_qual	
- bsmt_cond
- bsmt_exposure
- bsmtfin_type_1
- bsmtfin_type_2
- heating_qc
- functional
- fireplace_qu
- garage_finish
- garage_qual
- garage_cond
- paved_drive
- fence
- kitchen_qual
- pool_qc
- utilities
- electrical
- land_slope



In [74]:
#Transforming categorical to numerical
train['mas_vnr_area'] = train['mas_vnr_area'].astype(int)
train['garage_yr_blt'] = train['garage_yr_blt'].astype(int)

#Transforming numerical to categorical
train['ms_subclass'] = train['ms_subclass'].astype(int)

### Ordinal Variables Mapping for train set

For ordinal categories, we will have to map it to the numerical value of sequential numbers starting from 0 to be in line with the rating which mostly indicates the quality level. Therefore sequential number starting from 0 will be for the lowest rating will make sense.

In [75]:
#Lot Shape will be assigned numeric values according to irregularity 
train['lot_shape'] = train['lot_shape'].map({'Reg':0,'IR1':1,'IR2':2,'IR3':3})

In [76]:
#Overall Qual and Overall Cond are ints no numeric assignment required

In [77]:
train['exter_qual'].value_counts()

TA    1247
Gd     697
Ex      81
Fa      26
Name: exter_qual, dtype: int64

In [78]:
train['exter_cond'].value_counts()

TA    1778
Gd     215
Fa      49
Ex       7
Po       2
Name: exter_cond, dtype: int64

In [79]:
train['bsmt_qual'].value_counts()

TA    887
Gd    864
Ex    184
Fa     60
Na     55
Po      1
Name: bsmt_qual, dtype: int64

In [80]:
train['bsmt_cond'].value_counts()

TA    1834
Gd      89
Fa      65
Na      55
Po       5
Ex       3
Name: bsmt_cond, dtype: int64

In [81]:
#map a scale to Exter_qual, exter_cond, bsmt_qual, bsmt_cond and heating_qc category TA being the highest number, will be 0 and anything below should be negative.

for i in ['exter_qual','exter_cond', 'bsmt_qual', 'bsmt_cond', 'heating_qc', 'fireplace_qu', 'garage_qual', 'garage_cond', 'kitchen_qual', 'pool_qc']:
    train[i] = train[i].map({'Ex':5, 'Gd': 4, 'TA': 3, 'Fa':2, 'Po': 1, 'Na': 0 })

In [82]:
train['bsmt_exposure'].value_counts()

No    1339
Av     288
Gd     203
Mn     163
Na      58
Name: bsmt_exposure, dtype: int64

In [83]:
train['bsmt_exposure'] = train['bsmt_exposure'].map({'No':1, 'Mn': 2, 'Av':3, 'Gd':4, 'Na':0})

In [84]:
train['bsmtfin_type_1'].value_counts()

GLQ    615
Unf    603
ALQ    293
BLQ    200
Rec    183
LwQ    102
Na      55
Name: bsmtfin_type_1, dtype: int64

In [85]:
train['bsmtfin_type_2'].value_counts()

Unf    1749
Rec      80
LwQ      60
Na       56
BLQ      48
ALQ      35
GLQ      23
Name: bsmtfin_type_2, dtype: int64

In [86]:
#Setting ALQ as zero since it is the average and anything below is captured as somewhat negative

for i in ['bsmtfin_type_2','bsmtfin_type_1']:
    train[i] = train[i].map({'Unf':1, 'Rec': 3, 'LwQ': 2, 'BLQ':4, 'ALQ': 5, 'GLQ': 6, 'Na':0})

In [87]:
train['functional'].value_counts()

Typ     1915
Min2      42
Min1      42
Mod       29
Maj1      12
Maj2       7
Sev        2
Sal        2
Name: functional, dtype: int64

In [88]:
train['functional'] = train['functional'].map({'Typ': 7, 'Min1': 6, 'Min2':5, 'Mod':4, 'Maj1':3, 'Maj2':2, 'Sev':1, 'Sal':0})

In [89]:
train['utilities'].value_counts()

AllPub    2049
NoSewr       1
NoSeWa       1
Name: utilities, dtype: int64

In [90]:
train['utilities'] = train['utilities'].map({'AllPub':3,'NoSewr':2,'NoSeWa':1,'ELO':0})

In [91]:
train['land_slope'].value_counts()

Gtl    1953
Mod      88
Sev      10
Name: land_slope, dtype: int64

In [92]:
train['land_slope'] = train['land_slope'].map({'Gtl':2, 'Mod': 1, 'Sev':0})

In [93]:
train['electrical'].value_counts()

SBrkr    1868
FuseA     140
FuseF      35
FuseP       7
Mix         1
Name: electrical, dtype: int64

In [94]:
train['electrical'] = train['electrical'].map({'SBrkr': 4, 'FuseA':3 , 'FuseF': 2 , 'FuseP': 1, 'Mix': 0})

In [95]:
train['paved_drive'].value_counts()

Y    1861
N     151
P      39
Name: paved_drive, dtype: int64

In [96]:
train['paved_drive'] = train['paved_drive'].map({'Y': 2, 'P': 1, 'N': 0})

In [97]:
train['garage_finish'].value_counts()

Unf    849
RFn    579
Fin    509
Na     114
Name: garage_finish, dtype: int64

In [98]:
train['garage_finish'] = train['garage_finish'].map({'Fin':3, 'RFn':2, 'Unf':1, 'Na':0})

In [99]:
#Check for null values
train.isnull().sum().sum()

0

In [100]:
#saves a cleaned training data
train.to_csv('../datasets/train_clean.csv', index=False)

### Read in test data for data cleaning

We will now perform the same data cleaning steps on the test set as train set for alignment. 

In [101]:
test = pd.read_csv('../datasets/test.csv')

In [102]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,NaN,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,NaN,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [103]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               879 non-null    int64  
 1   PID              879 non-null    int64  
 2   MS SubClass      879 non-null    int64  
 3   MS Zoning        879 non-null    object 
 4   Lot Frontage     719 non-null    float64
 5   Lot Area         879 non-null    int64  
 6   Street           879 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        879 non-null    object 
 9   Land Contour     879 non-null    object 
 10  Utilities        879 non-null    object 
 11  Lot Config       879 non-null    object 
 12  Land Slope       879 non-null    object 
 13  Neighborhood     879 non-null    object 
 14  Condition 1      879 non-null    object 
 15  Condition 2      879 non-null    object 
 16  Bldg Type        879 non-null    object 
 17  House Style     

In [104]:
test.shape

(879, 80)

### Fill in Null Information

In [105]:
#List and sort the columns with missing values
null_columns1=test.columns[test.isnull().any()]
test[null_columns1].isnull().sum().sort_values(ascending=False)

Pool QC           875
Misc Feature      838
Alley             821
Fence             707
Fireplace Qu      422
Lot Frontage      160
Garage Yr Blt      45
Garage Finish      45
Garage Qual        45
Garage Cond        45
Garage Type        44
BsmtFin Type 2     25
BsmtFin Type 1     25
Bsmt Exposure      25
Bsmt Cond          25
Bsmt Qual          25
Mas Vnr Area        1
Mas Vnr Type        1
Electrical          1
dtype: int64

In [106]:
#Replacing null values with Na
cols = ['Pool QC',
        'Misc Feature', 
        'Alley', 
        'Fence', 
        'Fireplace Qu', 
        'Garage Type', 
        'Garage Finish',
        'Garage Qual',
        'Garage Cond',
        'BsmtFin Type 2',
        'BsmtFin Type 1',
        'Bsmt Exposure',
        'Bsmt Qual',
        'Bsmt Cond',
        'Mas Vnr Type',
        'Electrical'
       ]
for i in [cols]:
    test[i] = test[i].fillna('Na')

In [107]:
#Fill null with 0
test['Garage Yr Blt'] = test['Garage Yr Blt'].fillna('0')

In [108]:
#Change Mas Vnr Area to 0
test['Mas Vnr Area'] = test['Mas Vnr Area'].fillna('0')

In [110]:
#Fill 'Lot Frontage' with the mean
test['Lot Frontage'] = train['lot_frontage'].fillna(test['Lot Frontage'].mean())

In [111]:
#Check that all null values are resolved
test.columns[test.isnull().any()]

Index([], dtype='object')

In [112]:
#Change columns names to lower case and replace space with underscore
test.columns = test.columns.str.lower().str.replace(' ','_')

### Ordinal  Variables Mapping for test set

In [113]:
#Lot Shape will be assigned numeric values according to irregularity 
test['lot_shape'] = test['lot_shape'].map({'Reg':0,'IR1':1,'IR2':2,'IR3':3})

In [114]:
#map a scale to Exter_qual, exter_cond, bsmt_qual, bsmt_cond and heating_qc category from a scale of 0 to 5 and 5 being the best. 

for i in ['exter_qual','exter_cond', 'bsmt_qual', 'bsmt_cond', 'heating_qc', 'fireplace_qu', 'garage_qual', 'garage_cond', 'kitchen_qual', 'pool_qc']:
    test[i] = test[i].map({'Ex':5, 'Gd': 4, 'TA': 3, 'Fa':2, 'Po': 1, 'Na': 0 })

In [115]:
#Mapping the categories 
for i in ['bsmtfin_type_2','bsmtfin_type_1']:
    test[i] = test[i].map({'Unf':1, 'Rec': 3, 'LwQ': 2, 'BLQ':4, 'ALQ': 5, 'GLQ': 6, 'Na':0})

In [116]:
#Mapping the categories for Function
test['functional'] = test['functional'].map({'Typ': 7, 'Min1': 6, 'Min2':5, 'Mod':4, 'Maj1':3, 'Maj2':2, 'Sev':1, 'Sal':0})

In [117]:
#Mapping the categories for Utilities
test['utilities'] = test['utilities'].map({'AllPub':3,'NoSewr':2,'NoSeWa':1,'ELO':0})

In [118]:
#Mapping the categories for Electrical
test['land_slope'] = test['land_slope'].map({'Gtl':2, 'Mod': 1, 'Sev':0})

In [119]:
test['electrical'].unique()

array(['FuseP', 'SBrkr', 'FuseA', 'FuseF', 'Na'], dtype=object)

In [120]:
test['electrical'] = test['electrical'].map({'SBrkr': 4, 'FuseA':3 , 'FuseF': 2 , 'FuseP': 1, 'Na': 0})

In [121]:
test['paved_drive'] = test['paved_drive'].map({'Y': 2, 'P': 1, 'N': 0})

In [122]:
test['garage_finish'] = test['garage_finish'].map({'Fin':3, 'RFn':2, 'Unf':1, 'Na':0})

In [123]:
test['bsmt_exposure'] = test['bsmt_exposure'].map({'No':1, 'Mn': 2, 'Av':3, 'Gd':4, 'Na':0})

In [124]:
#Check for null values
test.isnull().sum().sum()

0

In [125]:
test.shape

(879, 80)

In [126]:
test.to_csv('../datasets/test_clean1.csv', index=False)